In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to loadimport numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
train=pd.read_csv('../input/lish-moa/train_features.csv')
test=pd.read_csv('../input/lish-moa/test_features.csv')
train_tar_scored=pd.read_csv('../input/lish-moa/train_targets_scored.csv')
train_tar_nonscored=pd.read_csv('../input/lish-moa/train_targets_nonscored.csv')
samp=pd.read_csv('../input/lish-moa/sample_submission.csv')

In [ ]:
print(train_tar_scored.shape)
print(train.shape)
print(test.shape)
print(samp.shape)
print(train.columns)
print(train.cp_type.value_counts())
print(train.cp_time.value_counts())
print(train.cp_dose.value_counts())

In [ ]:
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import KFold
from sklearn.preprocessing import LabelEncoder
import math
import numpy as np

def custom_loss(y_real,y_pred):
    res=0
    for i in range(y_real.shape[0]):
        for j in range(y_real.shape[1]):
            if(y_real.iloc[i][j]==0):
                res+=-(math.log(y_pred.iloc[i][j]))
            else:
                res+=-(math.log(1-y_pred.iloc[i][j]))
    return res/(y_pred.shape[0]*y_pred.shape[1])

type_enc=LabelEncoder()
dose_enc=LabelEncoder()
seed = 5
np.random.seed(seed)
X = train.iloc[:,4:877]
Y = train_tar_scored.iloc[:,1:208]
#X['cp_type']=type_enc.fit_transform(X['cp_type']).astype('int64')
#X['cp_dose']=dose_enc.fit_transform(X['cp_dose']).astype('int64')
X=(X-X.mean())/X.std()
print(X.dtypes)
print(Y.dtypes)
kfold = KFold(n_splits=10, shuffle=True, random_state=seed)

model=keras.Sequential([
    layers.Dense(1024, activation='relu', input_shape=[872]),
    layers.Dense(712,activation='sigmoid'),
    layers.Dense(498,activation='relu'),
    layers.Dense(206, activation='sigmoid'),
])
model.compile(loss='binary_crossentropy', optimizer='adam',metrics=['accuracy'])
model.fit(X,Y,epochs=8, batch_size=128, verbose=1)

print('Done')

In [ ]:

Z=test.iloc[:,4:877]
res=pd.DataFrame(model.predict(Z))
res.insert(0,'id',test.iloc[:,0],True)
res.columns=train_tar_scored.columns
print(res)
res.to_csv('submission.csv',index=False)

In [ ]:
model.evaluate(X,Y,verbose=1)

In [ ]:
pd.read_csv('./submission.csv').head()